In [3]:
!git clone https://github.com/mateoKutnjak/pose_est.git
import sys
sys.path.append('/content/pose_est')

Cloning into 'pose_est'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 2), reused 11 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls "/content/drive/My Drive"

'1. Pretraživanja.gslides'
 annotations
'Colab Notebooks'
 CV-Europass-20190209-EN-2.pdf
 Europass
 images
'Napoleon Hill - Think and Grow Rich - Unabridged Version'
 src.zip


In [6]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

TPU address is grpc://10.97.93.218:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 3261560686400638842),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 8547576070274266980),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 9311678404329993193),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 13976680515564298405),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9095568487838018993),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 1635696519960933009),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 18061825034643991774),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 101235605158177391),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 183480921527953

In [7]:
from pose_est.dataset import MPII_dataset
from pose_est.models import HourglassModel
import tensorflow as tf
import keras.backend as k
import argparse

/content/pose_est/dataset.py:32: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(input_shape), 3, "Input must have 3 dimensions")
/content/pose_est/dataset.py:33: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(output_shape), 3, "Output must have 3 dimensions")
/content/pose_est/dataset.py:34: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (input_shape[-1], 3, "Input channels dimension must be three (RGB image)")
/content/pose_est/dataset.py:35: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (output_shape[-1], self.joints_num, "Output channels dimension must be same as joints number")
Using TensorFlow backend.


In [0]:
IMAGES_DIR = '/content/drive/My Drive/images'
ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/annotations/annotations.json'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8
STACKS_NUM = 2

In [0]:
mpii_dataset = MPII_dataset(
        images_dir=IMAGES_DIR,
        annots_json_filename=ANNOTATIONS_JSON_FILE,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
        type='train'
    )

In [0]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1.0

In [0]:
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
hg_model = HourglassModel(
    dataset=mpii_dataset,
    batch_size=BATCH_SIZE,
    stacks_num=STACKS_NUM,
)

In [13]:
hg_model.build()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
front_module_conv2d_7x7 (Conv2D (None, 128, 128, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 128, 128, 64) 256         front_module_conv2d_7x7[0][0]    
__________________________________________________________________________________________________
front_residual_1_conv_1x1_first (None, 128, 128, 64) 4160        batch_normalization_1[0][0]      
_____________________________________

In [0]:
hg_model.train()

A
Epoch 1/1
   8/2780 [..............................] - ETA: 30:18:36 - loss: 2.6932 - 0_head_conv_1x1_third-heatmap_loss: 1.7202 - 1_head_conv_1x1_third-heatmap_loss: 0.9730 - 0_head_conv_1x1_third-heatmap_acc: 0.0574 - 1_head_conv_1x1_third-heatmap_acc: 0.0741